In [ ]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# Imports
import numpy as np
import pandas as pd
import os
import json
import sys
import cortex
from scipy import stats
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


sys.path.append("{}/../../../utils".format(os.getcwd()))
from plot_utils import *
from surface_utils import load_surface
from pycortex_utils import get_rois, calculate_vertex_areas, data_from_rois, load_surface_pycortex
from maths_utils import linear_regression_surf, multipletests_surface

# Import data base
main_dir = '/Users/uriel/disks/meso_shared'
# main_dir = '/home/ulascombes//disks/meso_shared'
project_dir = 'RetinoMaps'
subjects = ['sub-01']
subject = 'sub-12'
format_ = 'fsnative'
tsv_dir ='{}/{}/derivatives/pp_data/{}/{}/prf/tsv'.format(main_dir, 
                                                                project_dir, 
                                                                subject,
                                                         format_)


with open('../../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
rois = analysis_info['rois']


In [ ]:
data = pd.read_table('{}/{}_css-all_derivatives.tsv'.format(tsv_dir,subject))
ecc_th = [0, 15]
size_th= [0.1, 20]
rsq_th = [0, 1]
pcm_th = [0,20]

In [ ]:

def combine_pdfs_vertically(pdf_files, output_path):
    # Créer un objet writer pour le document final
    writer = PdfWriter()

    # Initialiser les variables pour la largeur et la hauteur totale
    total_height = 0
    width = None

    # Parcourir chaque fichier PDF, ouvrir, lire et calculer les dimensions
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        page = reader.pages[0]
        if width is None:
            width = page.mediabox[2]  # La largeur du premier PDF
        total_height += page.mediabox[3]  # Ajoute la hauteur de chaque page

    # Définir les dimensions de la nouvelle page
    mediabox = (0, 0, width, total_height)

    # Ajouter les pages au nouveau PDF
    current_height = total_height
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        page = reader.pages[0]
        page_height = page.mediabox[3]
        
        # Translate the origin of the page to the correct position
        page.mediabox = mediabox
        page.cropbox = (0, current_height - page_height, width, current_height)
        writer.add_page(page)
        current_height -= page_height  # Déplacer l'origine vers le bas pour la prochaine page

    # Écrire le document final
    with open(output_path, "wb") as f:
        writer.write(f)


# heat map

In [ ]:
def gaus_2d(gauss_x, gauss_y, gauss_sd, screen_side, grain=200):
    """
    Generate 2D gaussian mesh
    
    Parameters
    ----------
    gauss_x : mean x gaussian parameter in dva (e.g. 1 dva)
    gauss_y : mean y gaussian parameter in dva (e.g. 1 dva)
    gauss_sd : sd gaussian parameter in dva (e.g. 1 dva)
    screen_side : mesh screen side (square) im dva (e.g. 20 dva from -10 to 10 dva)
    grain : grain resolution of the mesh in pixels (default = 100 pixels)
    
    Returns
    -------
    x : linspace x of the mesh
    y : linspace x of the mesh
    z : mesh_z values (to plot)
    
    """
    x = np.linspace(-screen_side/2, screen_side/2, grain)
    y = np.linspace(-screen_side/2, screen_side/2, grain)
    mesh_x, mesh_y = np.meshgrid(x,y) 
    
    gauss_z = 1./(2.*np.pi*gauss_sd*gauss_sd)*np.exp(-((mesh_x-gauss_x)**2./(2.*gauss_sd**2.)+(mesh_y-gauss_y)**2./(2.*gauss_sd**2.)))
    return x, y, gauss_z

In [ ]:
len(data)

In [ ]:

vert = data.prf_loo_r2.idxmax()

In [ ]:
grain=200
x_tot = np.zeros(grain) 
y_tot = np.zeros(grain) 
gauss_z_tot = np.zeros((grain,grain)) 
for vert in range(len(data)):
    x, y, gauss_z = gaus_2d(gauss_x=data.prf_x[vert],  
                        gauss_y=data.prf_y[vert], 
                        gauss_sd=data.prf_size[vert], 
                        screen_side=30, 
                        grain=grain)

    gauss_z_tot += gauss_z * data.prf_loo_r2[vert]
gauss_z_tot = (gauss_z_tot-gauss_z_tot.min())/(gauss_z_tot.max()-gauss_z_tot.min())

In [ ]:
gauss_z_tot

In [ ]:
fig=go.Figure()
fig.add_trace(go.Histogram2dContour(x=x, y=y, z=gauss_z_tot, colorscale='viridis', showscale=True))
fig.update_layout(height=800, width=800)

In [ ]:
fig=go.Figure()
fig.add_trace(go.Heatmap(x=x, y=y, z=gauss_z_tot,colorscale='viridis', showscale=True))
fig.update_layout(height=800, width=800)

In [ ]:
grain = 200
x_tot = np.zeros(grain) 
y_tot = np.zeros(grain) 
gauss_z_tot = np.zeros((grain, grain)) 

for vert in range(len(data)):
    x, y, gauss_z = gaus_2d(gauss_x=data['prf_x'][vert],  
                             gauss_y=data['prf_y'][vert], 
                             gauss_sd=data['prf_size'][vert], 
                             screen_side=30, 
                             grain=grain)

    gauss_z_tot += gauss_z * data['prf_loo_r2'][vert]

gauss_z_tot = (gauss_z_tot - gauss_z_tot.min()) / (gauss_z_tot.max() - gauss_z_tot.min())



In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram2dContour(x=x, y=y, z=gauss_z_tot, colorscale='viridis'))
fig.update_layout(height=800, width=800)

fig.show()

# figs

In [ ]:
fig = prf_violins_plot(data, subject,fig_height=1080, fig_width=1920, ecc_th=ecc_th, size_th=size_th, rsq_th=rsq_th, pcm_th=pcm_th)

In [ ]:
fig.show()
# fig.write_image("/Users/uriel/Downloads/{}_violins5.pdf".format(subject))

In [ ]:
fig2 = prf_ecc_size_plot(data, subject, fig_height=400, fig_width=800, ecc_th=ecc_th, size_th=size_th, rsq_th=rsq_th)

In [ ]:
fig2.show()

In [ ]:
figures, hemis = prf_polar_plot(data, subject, fig_height=300, fig_width=1920, ecc_th=ecc_th, size_th=size_th, rsq_th=rsq_th)

for i, (figure, hemi) in enumerate(zip(figures, hemis), start=1):

    figure.write_image('/Users/uriel/Downloads/{}_subplot_polar_{}.pdf'.format(subject,hemi))


In [ ]:
fig5 = prf_contralaterality_plot(data, subject, fig_height=300, fig_width=1920, ecc_th=ecc_th, size_th=size_th, rsq_th=rsq_th)

In [ ]:
fig5.show()

# fi6

In [ ]:
fig6 = prf_ecc_pcm_plot(data, subject, fig_height=400, fig_width=800, ecc_th=ecc_th, pcm_th=pcm_th, rsq_th=rsq_th)

In [ ]:
fig6.show()

In [ ]:
fig6.write_image("/Users/uriel/Downloads/{}_pcm.pdf".format(subject))

# Fig 7

In [ ]:
fig7 = categories_proportions_roi_plot(data, subject, fig_height=300, fig_width=1920)

In [ ]:
fig7.show()

In [ ]:
fig7.write_image("/Users/uriel/Downloads/{}percentage.pdf".format(subject))

In [ ]:
fig8 = surface_rois_categories_plot(data, subject, fig_height=1080, fig_width=1920)

In [ ]:
fig8.show()

In [ ]:
fig9 = surface_rois_all_categories_plot(data, subject, fig_height=1080, fig_width=1920,)


In [ ]:
fig9.show()